# Feature selection and feature engineering

This isn't based on a news article, exactly, it's from a paper. You can see the paper in `/data/`.

In [1]:
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std


pd.set_option("display.max_columns", 200)
pd.set_option("display.max_colwidth", 200)
%matplotlib inline

# Reading in our data

## Every single person in a crash

We'll start by reading in the list of people who were involved in an accident.

**Call this dataframe `people`.**

In [2]:
people_df = pd.read_csv('sources/combined-person-data.csv', dtype=str)
people_df.head()

,AIRBAG_DEPLOYED,ALCOHOL_TESTTYPE_CODE,ALCOHOL_TEST_CODE,BAC_CODE,CDL_FLAG,CLASS,CONDITION_CODE,DATE_OF_BIRTH,DRUG_TESTRESULT_CODE,DRUG_TEST_CODE,EJECT_CODE,EMS_UNIT_LABEL,EQUIP_PROB_CODE,FAULT_FLAG,INJ_SEVER_CODE,LICENSE_STATE_CODE,MOVEMENT_CODE,OCC_SEAT_POS_CODE,PED_LOCATION_CODE,PED_OBEY_CODE,PED_TYPE_CODE,PED_VISIBLE_CODE,PERSON_ID,PERSON_TYPE,REPORT_NO,SAF_EQUIP_CODE,SEX_CODE,VEHICLE_ID
0,1.0,NaN,0.0,NaN,N,C,0.0,1952-04-20 00:00:00,NaN,0.0,1.0,NaN,1.0,N,1,PA,NaN,NaN,NaN,NaN,NaN,NaN,48dd00ee-e033-47e7-ad1e-0b734020301b,D,AB4284000S,13.0,F,eb6aadb8-dacb-4744-a1a7-ab812c96f27f
1,1.0,NaN,0.0,NaN,N,NaN,0.0,1985-05-28 00:00:00,NaN,0.0,0.0,NaN,0.0,N,1,MD,NaN,NaN,NaN,NaN,NaN,NaN,166296bd-ffd3-4c16-aa74-4f4bf4139d8d,D,AB4313000X,13.0,F,b463eb20-2f01-4200-9d6f-b18888ce2593
2,1.0,NaN,0.0,NaN,N,C,0.0,1960-10-04 00:00:00,NaN,0.0,0.0,NaN,0.0,Y,1,MD,NaN,NaN,NaN,NaN,NaN,NaN,f3b2743f-fbc3-4345-9419-56a0ca29102c,D,AB4313000X,13.0,F,3c8629d0-d524-47c1-bfbc-b18e07f3087e
3,1.0,NaN,0.0,NaN,N,B,0.0,1971-05-28 00:00:00,NaN,0.0,1.0,NaN,0.0,N,1,MD,NaN,NaN,NaN,NaN,NaN,NaN,5bbe589b-a2db-4dbb-be9b-17a800d69a08,D,AB4313000Y,0.0,F,c4628cdb-f295-4a24-8a4b-653741ac6ae7
4,1.0,NaN,0.0,NaN,N,D,0.0,1955-04-23 00:00:00,NaN,0.0,0.0,NaN,1.0,N,1,SC,NaN,NaN,NaN,NaN,NaN,NaN,b914136f-5ecd-46bb-94ec-ff5d4136c3eb,D,AB4669001F,13.0,F,cdda1580-fd79-4358-8819-c2250f494591


In [3]:
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888283 entries, 0 to 888282
Data columns (total 28 columns):
AIRBAG_DEPLOYED          638348 non-null object
ALCOHOL_TESTTYPE_CODE    17730 non-null object
ALCOHOL_TEST_CODE        679193 non-null object
BAC_CODE                 10912 non-null object
CDL_FLAG                 662502 non-null object
CLASS                    424196 non-null object
CONDITION_CODE           676947 non-null object
DATE_OF_BIRTH            825345 non-null object
DRUG_TESTRESULT_CODE     4885 non-null object
DRUG_TEST_CODE           679193 non-null object
EJECT_CODE               871592 non-null object
EMS_UNIT_LABEL           141095 non-null object
EQUIP_PROB_CODE          871592 non-null object
FAULT_FLAG               677048 non-null object
INJ_SEVER_CODE           888283 non-null object
LICENSE_STATE_CODE       608961 non-null object
MOVEMENT_CODE            16691 non-null object
OCC_SEAT_POS_CODE        209090 non-null object
PED_LOCATION_CODE        16691

How often did each severity of injury show up? (e.g. not injured, non-incapacitating injury, etc)

In [4]:
people_df.INJ_SEVER_CODE.value_counts()

1    716806
2     82642
3     76801
4     10353
5      1681
Name: INJ_SEVER_CODE, dtype: int64

In [5]:
people_df['FATALITY'] = (people_df.INJ_SEVER_CODE == '5').astype(int)
people_df.FATALITY.value_counts()

0    886602
1      1681
Name: FATALITY, dtype: int64

## Working on Features

### Starting our analysis

We're going to run a regression on the impact of being **male vs female on crash fatalities**. Prepare a dataframe called `train_df` with the appropriate information in it.

* **Tip:** What column(s) are your input, and what is your output? Aka independent and dependent variables
* **Tip:** You'll need to convert your input column into something numeric, I suggest using `.replace`
* **Tip:** We aren't interested in the "Unknown" sex - either filtering or `np.nan` + `.dropna()` might be useful ways to get rid of those columns

In [6]:
people_df.SEX_CODE.value_counts()

M    460973
F    354854
U     11067
Name: SEX_CODE, dtype: int64

In [7]:
train_df = people_df[['FATALITY', 'SEX_CODE']]
train_df = train_df[(train_df['SEX_CODE'] != 'U')].reset_index(drop=True)
train_df.rename(columns={'SEX_CODE':'MALE'}, inplace=True)
train_df = train_df.dropna(axis=0, how='any')
train_df.MALE.replace({'M':1, 'F':0}, inplace=True)
train_df.shape

(815827, 2)

Confirm that your `train_df` has two columns and 815,827 rows.

> **Tip:** If you have more rows, make sure you dropped all of the rows with Unknown sex.
>
> **Tip:** If you have more columns, make sure you only have your input and output columns.

### Run your regression

See the effect of sex on whether the person's injuries are fatal or not. I want to see a result dataframe that includes:

* Feature name
* Coefficient
* Odds ratio

In [8]:
X = train_df.drop(columns='FATALITY')
y = train_df.FATALITY
clf = lm.LogisticRegression(C=1e9, solver='lbfgs', max_iter=4000) 
clf.fit(X,y)
results = pd.DataFrame(np.exp(clf.coef_), columns=X.columns)
results

,MALE
0,2.043798


In [9]:
feature_names = X.columns
coefficients = clf.coef_[0]
pd.DataFrame({
    'feature': feature_names,
    'coefficient:': coefficients,
    'odds ratio': np.exp(coefficients)
}).sort_values(by='odds ratio', ascending=False)

,feature,coefficient:,odds ratio
0,MALE,0.71481,2.043798


### Use words to interpret this result

## Adding more features

The actual crash data has more details - whether it was snowy/icy, whether it was a highway, etc. 

Read in `combined-crash-data.csv` and merge it with our people dataset. I'll save you a lookup: the `REPORT_NO` is what matches between the two.

In [10]:
crash_df = pd.read_csv('sources/combined-crash-data.csv', dtype=str)

In [11]:
merge_df = people_df
merge_df = merge_df.merge(crash_df, how='left', on='REPORT_NO')

In [12]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 888336 entries, 0 to 888335
Data columns (total 65 columns):
AIRBAG_DEPLOYED          638375 non-null object
ALCOHOL_TESTTYPE_CODE    17732 non-null object
ALCOHOL_TEST_CODE        679240 non-null object
BAC_CODE                 10912 non-null object
CDL_FLAG                 662547 non-null object
CLASS                    424215 non-null object
CONDITION_CODE           676992 non-null object
DATE_OF_BIRTH            825393 non-null object
DRUG_TESTRESULT_CODE     4885 non-null object
DRUG_TEST_CODE           679240 non-null object
EJECT_CODE               871643 non-null object
EMS_UNIT_LABEL           141110 non-null object
EQUIP_PROB_CODE          871643 non-null object
FAULT_FLAG               677095 non-null object
INJ_SEVER_CODE           888336 non-null object
LICENSE_STATE_CODE       609002 non-null object
MOVEMENT_CODE            16693 non-null object
OCC_SEAT_POS_CODE        209096 non-null object
PED_LOCATION_CODE        16693

### Examining more possible features

How often was it wet, dry, snowy, icy, etc? **What was the most common condition?**

In [13]:
merge_df.SURF_COND_CODE.value_counts()

2.0     619569
1.0     147803
0.0      30921
3.0       9418
4.0       9111
99.0      3616
6.03      1697
88.0      1064
5.0        447
7.01       258
9.88       141
8.05        36
Name: SURF_COND_CODE, dtype: int64

Do you feel that a **Dry** road condition should be the average of **Wet** and **Snow?**

The answer to that should be *no*, which means we can't use this data as numeric data. We want a different coefficient for each of these - I want to know the impact of dry, the impact of wet, the impact of snow, all separately.

Start by **replacing each code with a proper description.** I'll even include them here:

* `00` - Not Applicable
* `01`	- Wet
* `02`	- Dry
* `03`	- Snow
* `04`	- Ice
* `05`	- Mud, Dirt, Gravel
* `06`	- Slush
* `07`	- Water (standing/moving)
* `08`	- Sand
* `09`	- Oil
* `88`	- Other
* `99`	- Unknown

But watch out, pandas read the column in as numbers so they might have come through a little differently than their codes.

In [14]:
merge_df.SURF_COND_CODE.replace({'0.0':'Not_Applicable',
                                 '1.0':'Wet',
                                 '2.0':'Dry',
                                 '3.0':'Snow',
                                 '4.0':'Ice',
                                 '5.0':'Mud_Dirt_Gravel',
                                 '6.03':'Slush',
                                 '7.01':'Water_standing_moving',
                                 '8.05':'Sand',
                                 '9.88':'Oil',
                                 '88.0':'Other',
                                 '99.0':'Unknown'
                                }, inplace=True)

Confirm you have 147,803 wet, and a few codes you can't understand, like `6.03` and `7.01`.

In [15]:
merge_df.SURF_COND_CODE.value_counts()

Dry                      619569
Wet                      147803
Not_Applicable            30921
Snow                       9418
Ice                        9111
Unknown                    3616
Slush                      1697
Other                      1064
Mud_Dirt_Gravel             447
Water_standing_moving       258
Oil                         141
Sand                         36
Name: SURF_COND_CODE, dtype: int64

Replace the codes you don't understand with `Other`.

In [16]:
merge_df.SURF_COND_CODE.replace({'Slush':'Other',
                                 'Water_standing_moving':'Other',
                                 'Sand':'Other',
                                 'Oil':'Other'
                                }, inplace=True)

Confirm you have 3,196 'Other'.

In [17]:
merge_df.SURF_COND_CODE.value_counts()

Dry                619569
Wet                147803
Not_Applicable      30921
Snow                 9418
Ice                  9111
Unknown              3616
Other                3196
Mud_Dirt_Gravel       447
Name: SURF_COND_CODE, dtype: int64

## One-hot encoding

We're going to use `pd.get_dummies` to build a variable you'll call `surf_dummies`. Each surface condition should be a `0` or `1` as to whether it was that condition (dry, icy, wet, etc).

Use a `prefix=` so we know they are **surface** conditions.

You'll want to drop the column you'll use as the reference category.

**Before we do this: which column works best as the reference?**

Now build your `surf_dummies` variable.

In [18]:
dummies_surf = pd.get_dummies(merge_df.SURF_COND_CODE, prefix='surf').drop('surf_Dry', axis=1)
dummies_surf.head()

,surf_Ice,surf_Mud_Dirt_Gravel,surf_Not_Applicable,surf_Other,surf_Snow,surf_Unknown,surf_Wet
0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0


Confirm your `surf_dummies` looks roughly like this:

|surface_Ice|Surce_Mud, Dirt, Gravel|surface_Not Applicable|...|surface_Wet|
|---|---|---|---|---|
|0|0|0|...|0|
|0|0|0|...|0|
|0|0|1|...|0|
|0|0|1|...|0|
|0|0|0|...|1|

## Another regression

Let's run another regression to see the impact of both **sex and surface condition** on fatalities.

### Build your `train_df`

To build your `train_df`, I recommend doing it either of these two ways:

```python
train_df = pd.DataFrame({
    'col1': merged.col1,
    'col2': merged.col2,
    'col3': merged.col3,
})
train_df = train_df.join(surf_dummies)
train_df = train_df.dropna()
```

or like this:

```python
train_df = train_df[['col1','col2','col3']].copy()
train_df = train_df.join(surf_dummies)
train_df = train_df.dropna()
```

The second one is shorter, but the first one makes it easier to use comments to remove columns later.


In [19]:
train_df = pd.DataFrame({
    'Fatality': merge_df.FATALITY,
    'Male': merge_df.SEX_CODE,
})
train_df = train_df.join(dummies_surf)
train_df = train_df.dropna(axis=0, how='any')
train_df = train_df[(train_df['Male'] != 'U')].reset_index(drop=True)
train_df.Male.replace({'M':1, 'F':0}, inplace=True)
train_df.head()

,Fatality,Male,surf_Ice,surf_Mud_Dirt_Gravel,surf_Not_Applicable,surf_Other,surf_Snow,surf_Unknown,surf_Wet
0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0
2,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0


### Run your regression and check your odds ratios

Actually no, wait, first - what kind of surface do you think will have the **highest fatality rate?**

In [20]:
X = train_df.drop(columns='Fatality')
y = train_df.Fatality
clf = lm.LogisticRegression(C=1e9, solver='lbfgs', max_iter=4000) 
clf.fit(X,y)
results = pd.DataFrame(np.exp(clf.coef_), columns=X.columns)
results

,Male,surf_Ice,surf_Mud_Dirt_Gravel,surf_Not_Applicable,surf_Other,surf_Snow,surf_Unknown,surf_Wet
0,2.046188,0.571557,3.057411,0.481106,1.812459,0.519835,1.404368,0.905202


In [21]:
feature_names = X.columns
coefficients = clf.coef_[0]
pd.DataFrame({
    'feature': feature_names,
    'coefficient:': coefficients,
    'odds ratio': np.exp(coefficients)
}).sort_values(by='odds ratio', ascending=False)

,feature,coefficient:,odds ratio
2,surf_Mud_Dirt_Gravel,1.117569,3.057411
0,Male,0.715979,2.046188
4,surf_Other,0.594685,1.812459
6,surf_Unknown,0.339587,1.404368
7,surf_Wet,-0.099597,0.905202
1,surf_Ice,-0.559390,0.571557
5,surf_Snow,-0.654243,0.519835
3,surf_Not_Applicable,-0.731667,0.481106


Confirm your `train_df` has 815,843 rows and 9 columns.

* **Tip:** When you run your regression, if you get an error about not knowing what to do with `U`, it's because you didn't convert your sex to numbers (or if you did, you didn't do it in your original dataframe)

In [22]:
train_df.shape

(815876, 9)

**Is this what you expected?** Why do you think this result might be the case?

# More features: Vehicles

Maybe the car they're in is related to the car they were in. Luckily, we have this information - **read in `combined_vehicle_data` as `vehicles`.**

In [23]:
vehicles_df = pd.read_csv('sources/combined-vehicle-data.csv', dtype=str)
vweights_df = pd.read_csv('sources/vins_and_weights.csv', dtype=str)

## Weights of those cars

The car weights are stored in **another file** since the info had to come from an API. I looked up the VINs - vehicle identification numbers - in a government database to try to get data for each of them.

**Read them and build a new dataframe that is both the vehicle data along with their weights.** You can call it `vehicles` since you don't need the original weightless vehicle data any more.

In [24]:
vehicles_df = vehicles_df.merge(vweights_df, how='inner', left_on='VIN_NO', right_on='VIN')
vehicles_df.drop('VIN', axis = 1)
vehicles_df = vehicles_df.rename({'Make':'VEH_MAKE2','Model':'VEH_MODEL2','ModelYear':'VEH_YEAR2','weight':'WEIGHT'}, axis=1)

In [25]:
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534436 entries, 0 to 534435
Data columns (total 35 columns):
AREA_DAMAGED_CODE1           514451 non-null object
AREA_DAMAGED_CODE2           313039 non-null object
AREA_DAMAGED_CODE3           202859 non-null object
AREA_DAMAGED_CODE_IMP1       530276 non-null object
AREA_DAMAGED_CODE_MAIN       528315 non-null object
BODY_TYPE_CODE               534436 non-null object
COMMERCIAL_FLAG              534436 non-null object
CONTI_DIRECTION_CODE         472061 non-null object
CV_BODY_TYPE_CODE            1321 non-null object
DAMAGE_CODE                  534436 non-null object
DRIVERLESS_FLAG              534436 non-null object
FIRE_FLAG                    455452 non-null object
GOING_DIRECTION_CODE         472061 non-null object
GVW_CODE                     1321 non-null object
HARM_EVENT_CODE              534436 non-null object
HAZMAT_SPILL_FLAG            337 non-null object
HIT_AND_RUN_FLAG             534436 non-null object
HZM_NUM     

Confirm that your combined `vehicles` dataset should have 534,436 rows and 35 columns. And yes, that's less than we were working with before - you haven't combined it with the people/crashes dataset yet.

In [26]:
vehicles_df.shape

(534436, 35)

### Filter your data

We only want vehicles that are "normal" - somewhere between 1500 and 6000 pounds. Filter your vehicles to only include those in that weight range.

In [27]:
vehicles_df.WEIGHT = vehicles_df.WEIGHT.astype(float).astype(int)
vehicles_df = vehicles_df[(vehicles_df.WEIGHT >= 1500) & (vehicles_df.WEIGHT <= 6000)].reset_index(drop=True)

Confirm that you have 532,370 vehicles in the dataset.

In [28]:
vehicles_df.shape

(532370, 35)

## Add this vehicle information to your merged data

Now we'll have a dataframe that contains information on:

* The people themselves and their injuries
* The crash
* The vehicles

Every person came with a `VEHICLE_ID` column that is the vehicle they were in. You'll want to merge on that.

In [29]:
merge_df = merge_df.merge(vehicles_df, how='inner', on='VEHICLE_ID')

Confirm you have 99 columns and 616,212 rows. **That is a lot of possible features!**

In [30]:
merge_df.shape

(616242, 99)

## Another regression, because we can't get enough

Build another `train_df` and run another regression about **how car weight impacts the chance of fatalities**. You'll want to confirm that your dataset has 616,212 and 2 columns.

In [31]:
train_df = pd.DataFrame({
    'Fatality': merge_df.FATALITY,
    'Weight': merge_df.WEIGHT,
})
train_df = train_df.dropna(axis=0, how='any')
train_df.shape

(616242, 2)

In [32]:
X = train_df.drop(columns='Fatality')
y = train_df.Fatality
clf = lm.LogisticRegression(C=1e9, solver='lbfgs', max_iter=4000) 
clf.fit(X,y)
results = pd.DataFrame(np.exp(clf.coef_), columns=X.columns)
results

,Weight
0,0.999851


In [33]:
feature_names = X.columns
coefficients = clf.coef_[0]
pd.DataFrame({
    'feature': feature_names,
    'coefficient:': coefficients,
    'odds ratio': np.exp(coefficients)
}).sort_values(by='odds ratio', ascending=False)

,feature,coefficient:,odds ratio
0,Weight,-0.000149,0.999851


**Can you translate that into plain English?** Remember weight is in **pounds**.

I feel like pounds isn't the best measure for something like this. Remember how we had to adjust percentages with AP and life expecntancy, and then change around the way we said things? It sounded like this:

> Every 10% increase in unemployment translates to a year and a half loss of life expectancy

Instead of every single pound, maybe we could do every... some other number of pounds? One hundred? One thousand?

**Run another regression with weight in thousands of pounds.** Get another odds ratio. Give me another sentence English.

In [34]:
train_df.Weight = train_df.Weight / 1000

In [35]:
X = train_df.drop(columns='Fatality')
y = train_df.Fatality
clf = lm.LogisticRegression(C=1e9, solver='lbfgs', max_iter=4000) 
clf.fit(X,y)
results = pd.DataFrame(np.exp(clf.coef_), columns=X.columns)

In [36]:
feature_names = X.columns
coefficients = clf.coef_[0]
pd.DataFrame({
    'feature': feature_names,
    'coefficient:': coefficients,
    'odds ratio': np.exp(coefficients)
}).sort_values(by='odds ratio', ascending=False)

,feature,coefficient:,odds ratio
0,Weight,-0.149477,0.861159


In [37]:
# Every thousand pounds heavier your car is increase translates to a 15% decrease in fatalities

# Two-car accidents, struck and striker

Here's the thing, though: **it isn't just the weight of your car.** It's the weight of both cars! If I'm in a big car and I have a wreck with a smaller car, it's the smaller car that's in trouble.

To get that value, we need to do some **feature engineering**, some calculating of *new* variables from our *existing* variables.

We need to jump through some hoops to do that.

##  Two-car accidents

First we're going to count how many vehicles were in each accident. Since we're looking to compare the weight of two cars hitting each other, **we're only going to want crashes with only two cars.**

In [38]:
carcounts_df = vehicles_df.REPORT_NO.value_counts()
carcounts_df.head(10)

CB53480003      28
MDTA1229000H    26
AS0286000B      14
CE4636002C      14
MSP67460095     13
MSP6063006C     13
CC0237003W      12
AE5607002W      12
MSP63070049     12
AC1811000Q      11
Name: REPORT_NO, dtype: int64

By using `.value_counts` I can see how many cars were in each crash, and now I'm going to filter to get a list of all of the ones with two vehicles.

In [39]:
twocar_report_nos = carcounts_df[carcounts_df == 2].index
twocar_report_nos

Index(['ADG540000D', 'MSP4876002M', 'ZVW252000G', 'DD5595004M', 'MSP57630039',
       'DJ01090003', 'AC1639000N', 'ZR03090011', 'DA38870008', 'CB5047000Q',
       ...
       'BW01370016', 'MSP6505003Z', 'ADJ2130052', 'MSP5331000G', 'AE4823000N',
       'GPDM65000R', 'MSP6059009R', 'MSP59520029', 'ADJ5300012', 'ADJ662000K'],
      dtype='object', length=137214)

And now we'll filter my vehicles so we only have those that were in two-vehicle crashes.

In [40]:
vehicles2x_df = vehicles_df[vehicles_df.REPORT_NO.isin(twocar_report_nos)]

### Struck and striker

To do the math correctly, we need both the risk of someone dying in the smaller car _and_ the risk of someone dying in the bigger car. To do this we need to separate our cars into two groups:

* The 'struck' vehicle: did the person die inside?
* The 'striker' vehicle: how much heavier was it than the struck car?

But we don't know which car was which, so we have to try out both versions - pretending car A was the striker, then pretending car B was the striker. It's hard to explain, but you can read `Pounds That Kill - The External Costs of Vehicle Weight.pdf` for more details on how it works.

In [41]:
cars_1 = vehicles2x_df.drop_duplicates(subset='REPORT_NO', keep='first')
cars_2 = vehicles2x_df.drop_duplicates(subset='REPORT_NO', keep='last')

In [42]:
cars_merged_1 = cars_1.merge(cars_2, on='REPORT_NO', suffixes=['_striker', '_struck'])
cars_merged_2 = cars_2.merge(cars_1, on='REPORT_NO', suffixes=['_striker', '_struck'])
vehicles_complete = pd.concat([cars_merged_1, cars_merged_2])
vehicles_complete.head()

,AREA_DAMAGED_CODE1_striker,AREA_DAMAGED_CODE2_striker,AREA_DAMAGED_CODE3_striker,AREA_DAMAGED_CODE_IMP1_striker,AREA_DAMAGED_CODE_MAIN_striker,BODY_TYPE_CODE_striker,COMMERCIAL_FLAG_striker,CONTI_DIRECTION_CODE_striker,CV_BODY_TYPE_CODE_striker,DAMAGE_CODE_striker,DRIVERLESS_FLAG_striker,FIRE_FLAG_striker,GOING_DIRECTION_CODE_striker,GVW_CODE_striker,HARM_EVENT_CODE_striker,HAZMAT_SPILL_FLAG_striker,HIT_AND_RUN_FLAG_striker,HZM_NUM_striker,MOVEMENT_CODE_striker,NUM_AXLES_striker,PARKED_FLAG_striker,REPORT_NO,SPEED_LIMIT_striker,TOWED_AWAY_FLAG_striker,TOWED_VEHICLE_CONFIG_CODE_striker,VEHICLE_ID_striker,VEH_MAKE_striker,VEH_MODEL_striker,VEH_YEAR_striker,VIN_NO_striker,VIN_striker,VEH_MAKE2_striker,VEH_MODEL2_striker,VEH_YEAR2_striker,WEIGHT_striker,AREA_DAMAGED_CODE1_struck,AREA_DAMAGED_CODE2_struck,AREA_DAMAGED_CODE3_struck,AREA_DAMAGED_CODE_IMP1_struck,AREA_DAMAGED_CODE_MAIN_struck,BODY_TYPE_CODE_struck,COMMERCIAL_FLAG_struck,CONTI_DIRECTION_CODE_struck,CV_BODY_TYPE_CODE_struck,DAMAGE_CODE_struck,DRIVERLESS_FLAG_struck,FIRE_FLAG_struck,GOING_DIRECTION_CODE_struck,GVW_CODE_struck,HARM_EVENT_CODE_struck,HAZMAT_SPILL_FLAG_struck,HIT_AND_RUN_FLAG_struck,HZM_NUM_struck,MOVEMENT_CODE_struck,NUM_AXLES_struck,PARKED_FLAG_struck,SPEED_LIMIT_struck,TOWED_AWAY_FLAG_struck,TOWED_VEHICLE_CONFIG_CODE_struck,VEHICLE_ID_struck,VEH_MAKE_struck,VEH_MODEL_struck,VEH_YEAR_struck,VIN_NO_struck,VIN_struck,VEH_MAKE2_struck,VEH_MODEL2_struck,VEH_YEAR2_struck,WEIGHT_struck
0,6.0,7.0,NaN,6.0,6.0,20.0,N,N,NaN,3,N,N,N,NaN,1.0,NaN,N,NaN,6.0,NaN,N,CB5190006B,55,N,0,0003b659-2785-4868-8877-0b786a284827,TOYT,TK,2011.0,5TFUY5F1XBX167340,5TFUY5F1XBX167340,TOYOTA,TUNDRA,2011,5480,3.0,5.0,6.0,6.0,6.0,23.08,N,N,NaN,5,N,N,N,NaN,1.0,NaN,N,NaN,6.0,NaN,N,55,Y,0,0e7da302-5f53-45c8-8b28-393cdc38171d,JEEP,CHEROKEE,2015.0,1C4RJFBGXFC751955,1C4RJFBGXFC751955,JEEP,GRAND CHEROKEE,2015,4545
1,8.0,NaN,NaN,8.0,8.0,2.0,N,W,NaN,2,N,NaN,W,NaN,1.0,NaN,N,NaN,8.0,NaN,N,ADJ4590035,5,N,0,00050484-d08f-4b6e-bc7e-9ec270e94660,HONDA,CIVIC,2015.0,2HGFG4A59FH702545,2HGFG4A59FH702545,HONDA,CIVIC,2015,2754,99.0,NaN,NaN,99.0,99.0,2.0,N,W,NaN,99,N,N,W,NaN,1.0,NaN,Y,NaN,1.0,NaN,N,5,N,0,3ebfe7fc-4832-414d-a5e8-391b65a2f280,HONDA,ACCORD,2013.0,1HGCT2B81DA005695,1HGCT2B81DA005695,HONDA,ACCORD,2013,3163
2,12.0,NaN,NaN,12.0,12.0,2.0,N,N,NaN,4,N,NaN,N,NaN,0.0,NaN,N,NaN,1.0,NaN,N,ADJ849000Z,10,N,0,00057af4-d848-4cee-b854-707f57581f4e,HONDA,ACCORD,2003.0,1HGCM66313A037175,1HGCM66313A037175,HONDA,ACCORD,2003,3023,12.0,NaN,NaN,12.0,12.0,2.0,N,NaN,NaN,3,Y,NaN,NaN,NaN,1.0,NaN,N,NaN,10.0,NaN,Y,0,N,0,4cf496cc-f537-447e-bb8e-3ef48c8c3ddf,HONDA,TK,2010.0,5J6RE3H73AL008454,5J6RE3H73AL008454,HONDA,CR-V,2010,3389
3,1.0,11.0,12.0,12.0,12.0,2.0,N,S,NaN,4,N,N,S,NaN,1.0,NaN,N,NaN,12.0,NaN,N,AE5207008Z,30,Y,0,00089d4a-7038-4693-9e02-b402676631af,FORD,4D,2016.0,1FADP3K28GL258987,1FADP3K28GL258987,FORD,FOCUS,2016,2932,11.0,12.0,NaN,12.0,12.0,23.08,N,N,NaN,4,N,N,N,NaN,1.0,NaN,N,NaN,1.0,NaN,N,30,Y,0,1e69b284-719f-429b-baae-6ae4976a546b,BUICK,TK,2002.0,3G5DB03E52S528316,3G5DB03E52S528316,BUICK,RENDEZVOUS,2002,4024
4,2.0,NaN,NaN,2.0,2.0,2.0,N,S,NaN,99,N,N,S,NaN,1.0,NaN,Y,NaN,1.0,NaN,N,ADJ859000Y,15,N,0,0010076b-0a45-45f3-8796-f387b39cd85d,HONDA,CIVIC,2009.0,2HGFA16689H357624,2HGFA16689H357624,HONDA,CIVIC,2009,2678,10.0,NaN,NaN,10.0,10.0,2.0,N,NaN,NaN,3,Y,N,NaN,NaN,1.0,NaN,N,NaN,10.0,NaN,Y,15,N,0,b14273cb-b3e5-42c7-af59-df069e8c2f0f,HONDA,CIVIC,2011.0,2HGFG1B87BH517469,2HGFG1B87BH517469,HONDA,CIVIC,2011,2703


In [43]:
vehicles_complete.shape

(274428, 69)

## Put people in their cars

Which car was each person in? We'll assign that now.

In [44]:
merged2x = people_df.merge(vehicles_complete, left_on='VEHICLE_ID', right_on='VEHICLE_ID_struck')
merged2x.head(3)

,AIRBAG_DEPLOYED,ALCOHOL_TESTTYPE_CODE,ALCOHOL_TEST_CODE,BAC_CODE,CDL_FLAG,CLASS,CONDITION_CODE,DATE_OF_BIRTH,DRUG_TESTRESULT_CODE,DRUG_TEST_CODE,EJECT_CODE,EMS_UNIT_LABEL,EQUIP_PROB_CODE,FAULT_FLAG,INJ_SEVER_CODE,LICENSE_STATE_CODE,MOVEMENT_CODE,OCC_SEAT_POS_CODE,PED_LOCATION_CODE,PED_OBEY_CODE,PED_TYPE_CODE,PED_VISIBLE_CODE,PERSON_ID,PERSON_TYPE,REPORT_NO_x,SAF_EQUIP_CODE,SEX_CODE,VEHICLE_ID,FATALITY,AREA_DAMAGED_CODE1_striker,AREA_DAMAGED_CODE2_striker,AREA_DAMAGED_CODE3_striker,AREA_DAMAGED_CODE_IMP1_striker,AREA_DAMAGED_CODE_MAIN_striker,BODY_TYPE_CODE_striker,COMMERCIAL_FLAG_striker,CONTI_DIRECTION_CODE_striker,CV_BODY_TYPE_CODE_striker,DAMAGE_CODE_striker,DRIVERLESS_FLAG_striker,FIRE_FLAG_striker,GOING_DIRECTION_CODE_striker,GVW_CODE_striker,HARM_EVENT_CODE_striker,HAZMAT_SPILL_FLAG_striker,HIT_AND_RUN_FLAG_striker,HZM_NUM_striker,MOVEMENT_CODE_striker,NUM_AXLES_striker,PARKED_FLAG_striker,REPORT_NO_y,SPEED_LIMIT_striker,TOWED_AWAY_FLAG_striker,TOWED_VEHICLE_CONFIG_CODE_striker,VEHICLE_ID_striker,VEH_MAKE_striker,VEH_MODEL_striker,VEH_YEAR_striker,VIN_NO_striker,VIN_striker,VEH_MAKE2_striker,VEH_MODEL2_striker,VEH_YEAR2_striker,WEIGHT_striker,AREA_DAMAGED_CODE1_struck,AREA_DAMAGED_CODE2_struck,AREA_DAMAGED_CODE3_struck,AREA_DAMAGED_CODE_IMP1_struck,AREA_DAMAGED_CODE_MAIN_struck,BODY_TYPE_CODE_struck,COMMERCIAL_FLAG_struck,CONTI_DIRECTION_CODE_struck,CV_BODY_TYPE_CODE_struck,DAMAGE_CODE_struck,DRIVERLESS_FLAG_struck,FIRE_FLAG_struck,GOING_DIRECTION_CODE_struck,GVW_CODE_struck,HARM_EVENT_CODE_struck,HAZMAT_SPILL_FLAG_struck,HIT_AND_RUN_FLAG_struck,HZM_NUM_struck,MOVEMENT_CODE_struck,NUM_AXLES_struck,PARKED_FLAG_struck,SPEED_LIMIT_struck,TOWED_AWAY_FLAG_struck,TOWED_VEHICLE_CONFIG_CODE_struck,VEHICLE_ID_struck,VEH_MAKE_struck,VEH_MODEL_struck,VEH_YEAR_struck,VIN_NO_struck,VIN_struck,VEH_MAKE2_struck,VEH_MODEL2_struck,VEH_YEAR2_struck,WEIGHT_struck
0,1.0,NaN,0.0,NaN,N,NaN,0.0,1985-05-28 00:00:00,NaN,0.0,0.0,NaN,0.0,N,1,MD,NaN,NaN,NaN,NaN,NaN,NaN,166296bd-ffd3-4c16-aa74-4f4bf4139d8d,D,AB4313000X,13.0,F,b463eb20-2f01-4200-9d6f-b18888ce2593,0,3.0,NaN,NaN,3.0,3.0,2.0,N,E,NaN,2,N,NaN,E,NaN,1.0,NaN,Y,NaN,2.0,NaN,N,AB4313000X,25,N,0,3c8629d0-d524-47c1-bfbc-b18e07f3087e,LEXU,TK,2004.0,2T2HA31U24C031048,2T2HA31U24C031048,LEXUS,RX,2004,3900,7.0,NaN,NaN,7.0,7.0,2.0,N,S,NaN,2,N,N,S,NaN,1.0,NaN,N,NaN,6.0,NaN,N,25,N,0,b463eb20-2f01-4200-9d6f-b18888ce2593,CHEVY,TK,2007.0,2CNDL13F576252855,2CNDL13F576252855,CHEVROLET,EQUINOX,2007,3776
1,1.0,NaN,0.0,NaN,N,C,0.0,1960-10-04 00:00:00,NaN,0.0,0.0,NaN,0.0,Y,1,MD,NaN,NaN,NaN,NaN,NaN,NaN,f3b2743f-fbc3-4345-9419-56a0ca29102c,D,AB4313000X,13.0,F,3c8629d0-d524-47c1-bfbc-b18e07f3087e,0,7.0,NaN,NaN,7.0,7.0,2.0,N,S,NaN,2,N,N,S,NaN,1.0,NaN,N,NaN,6.0,NaN,N,AB4313000X,25,N,0,b463eb20-2f01-4200-9d6f-b18888ce2593,CHEVY,TK,2007.0,2CNDL13F576252855,2CNDL13F576252855,CHEVROLET,EQUINOX,2007,3776,3.0,NaN,NaN,3.0,3.0,2.0,N,E,NaN,2,N,NaN,E,NaN,1.0,NaN,Y,NaN,2.0,NaN,N,25,N,0,3c8629d0-d524-47c1-bfbc-b18e07f3087e,LEXU,TK,2004.0,2T2HA31U24C031048,2T2HA31U24C031048,LEXUS,RX,2004,3900
2,1.0,NaN,0.0,NaN,N,C,0.0,1990-11-05 00:00:00,NaN,0.0,1.0,NaN,1.0,N,1,MD,NaN,NaN,NaN,NaN,NaN,NaN,21fff0f9-ed13-4b87-ac04-29d018aff9d4,D,AB5218001Y,13.0,F,4dea42c3-e02c-4c6b-8ea0-c2f8c17f147f,0,10.0,11.0,12.0,11.0,11.0,2.0,N,N,NaN,3,N,N,N,NaN,1.0,NaN,N,NaN,1.0,NaN,N,AB5218001Y,25,N,0,057a3e63-3009-4d07-b956-78f53a0ca992,LINC,TOWNCAR,2003.0,1LNHM82W53Y644084,1LNHM82W53Y644084,LINCOLN,TOWN CAR,2003,4000,9.0,10.0,11.0,11.0,11.0,23.08,N,S,NaN,3,N,N,W,NaN,1.0,NaN,N,NaN,12.0,NaN,N,25,N,0,4dea42c3-e02c-4c6b-8ea0-c2f8c17f147f,HYUN,SW,2010.0,5NMSG3AB9AH400386,5NMSG3AB9AH400386,HYUNDAI,SANTA FE,2010,3900


In [ ]:
merged2x.shape

# Add the crash details

You did this already! I'm going to do it for you. We're merging on `REPORT_NO_x` because there are so many `REPORT_NO` columns duplicated across our files that pandas started giving them weird names.

In [45]:
merged2x = merged2x.merge(crashes, left_on='REPORT_NO_x', right_on='REPORT_NO')
merged2x.head(3)

NameError: name 'crashes' is not defined

## Filter

We already filtered out vehicles by weight, so we don't have to do that again.

# Calculated features

I'm sure you forgot what all the features are, so we'll bring back whether there was a fatality or not

## Feature: Accident was fatal

In [ ]:
merged['had_fatality'] = (merged.INJ_SEVER_CODE == 5).astype(int)
merged.had_fatality.value_counts()

## Feature: Weight difference

**Remove everything missing weights for strikers or struck vehicles.** You might need to `merged.columns` to remind yourself what the column names are.

Confirm your dataset has 334,396 rows.

Create a new feature called `weight_diff` about how much heavier the striking car was compared to the struck car. **Make sure you've done the math correctly!**

### Feature adjustment

Make all of your weight columns in **thousands of pounds** instead of just in pounds. It'll help you interpret your results much better.

# Another regression!!!

**What is the impact of weight difference on fatality rate?** Create your `train_df`, drop missing values, run your regression, analyze your odds ratios.

Please translate your odds ratio into plain English.

## Adding in more features

How about speed limit? That's important, right? We can add the speed limit of the striking vehicle with `SPEED_LIMIT_striker`.

Can you translate the speed limit odds ratio into plain English?

### Feature engineering: Speed limits

Honestly, that's a pretty bad way to go about things. What's more fun is if we **translate speed limits into bins.**

First, we'll use `pd.cut` to assign each speed limit a category.

In [ ]:
speed_bins = [-np.inf, 10, 20, 30, 40, 50, np.inf]
merged['speed_bin'] = pd.cut(merged.SPEED_LIMIT_struck, bins=speed_bins)
merged[['SPEED_LIMIT_striker', 'speed_bin']].head(10)

Then we'll one-hot encode around 20-30mph speed limits.

In [ ]:
speed_dummies = pd.get_dummies(merged.speed_bin, 
                               prefix='speed').drop('speed_(20.0, 30.0]', axis=1)
speed_dummies.head()

## Running a regression

I like this layout for creating `train_df`, it allows us to easily add dummies and do a little replacing/encoding when we're building binary features like for sex.

> If the below gives you an error, it's because `SEX_CODE` is already a number. In that case, just remove `.replace({'M': 1, 'F': 0, 'U': np.nan })`.

In [ ]:
# Start with our normal features
train_df = pd.DataFrame({
    'weight_diff': merged.weight_diff,
    'sex': merged.SEX_CODE,#.replace({'M': 1, 'F': 0, 'U': np.nan }),
    'had_fatality': merged.had_fatality,
})
# Add the one-hot encoded features
train_df = train_df.join(speed_dummies)
train_df = train_df.join(surf_dummies)
# Drop missing values
train_df = train_df.dropna()
train_df.head()

Describe the impact of the different variables in simple language. What has the largest impact?

## Now you pick the features

Up above you have examples of:

* Creating features from numbers (speed limits)
* Creating features from 0/1 (sex)
* Creating features from binning numbers that are one-hot encoded (speed limit bins - `speed_bins`)
* Creating features from categories that are one-hot encoded (surface - `surf_dummies`

What else do you think matters? Try to plug in more features and see if you can get anything interesting.

> * **Hot tip:** The funniest/most interesting thing feature you can add is also the dumbest. Ask me about it in #algorithms if you end up getting down here.